In [44]:
from dotenv import load_dotenv
import openai
load_dotenv()
import os


os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")


In [45]:
from dotenv import load_dotenv
import os
import openai

# Load environment variables from .env file
load_dotenv()

# Set the OpenAI API key
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

# Verify the key is loaded
api_key = os.getenv("OPENAI_API_KEY")
if api_key:
    print("OpenAI API key loaded successfully.")
else:
    print("OpenAI API key not found. Please check your .env file.")


OpenAI API key loaded successfully.


In [47]:
from dotenv import load_dotenv
import os
import openai

# Load environment variables from .env file
load_dotenv()

# Set the OpenAI API key
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

# Verify the key is loaded
api_key = os.getenv("OPENAI_API_KEY")
if api_key:
    print("OpenAI API key loaded successfully.")
else:
    print("OpenAI API key not found. Please check your .env file.")


import pandas as pd
import re
import os
import json
import openai



OpenAI API key loaded successfully.


In [48]:
import pandas as pd
import re
import os
import json
import openai

# Set your OpenAI API key here or in environment variables
# openai.api_key = os.getenv("OPENAI_API_KEY")  



In [49]:
# Define GAAP + business categories
CATEGORIES = [
    "Travel",
    "Meals & Entertainment",
    "Office & Administration",
    "Marketing & Advertising",
    "Software & Subscriptions",
    "Utilities & Telecom",
    "Cost of Goods Sold",
    "Professional Services",
    "Human Resources / Payroll",
    "Miscellaneous / Other"
]

In [50]:
# Replace with your file path
file_path = "expenses.xlsx"
df = pd.read_excel(file_path)


In [51]:
df.head()

,Date,Description,Amount,Extended Details,Appears On Your Statement As,Address,City/State,Zip Code,Country,Reference,Category,CC Name,G/L Account No
0,2025-01-11,ONLINE PAYMENT - THANK YOU,-12021.51,ONLINE PAYMENT - THANK YOU,ONLINE PAYMENT - THANK YOU,NaN,NaN,NaN,NaN,'320250110186972138',NaN,DELTARESERVE,NaN
1,2025-01-11,ADOBE SYSTEMS Adobe SAN JOSE CA,21.64,567960456 www.adobe.com\nADOBE SYSTEMS Adobe...,ADOBE SYSTEMS Adobe SAN JOSE CA,801 N 34TH ST,SEATTLE\nWA,98103,UNITED STATES,'320250120216930217',Merchandise & Supplies-Mail Order,AMEXPLATINUM,64100.0
2,2025-01-11,GOOGLE *ISHARINGSOFT G.CO/HELPPAY# CA,16.04,A0YWKMVS DIGITAL GOODS: APPS\nGOOGLE *ISHAR...,GOOGLE *ISHARINGSOFT G.CO/HELPPAY# CA,1600 AMPHITHEATRE PKWY,MOUNTAIN VIEW\nCA,94043-1351,UNITED STATES,320250120218752716',Business Services-Internet Services,DELTARESERVE,64100.0
3,2025-01-12,UBER,28.32,FG9QDJ8X LM5CYPCR 10013\nUber Trip\n...,Uber Trip help.uber.com CA,1455 MARKET ST\n4TH FLOOR,SAN FRANCISCO\nCA,94103,UNITED STATES,'320250120213109403',Transportation-Taxis & Coach,DELTARESERVE,61130.0
4,2025-01-12,FACEBK *META VERIFIEFB.ME/CC CA,11.99,92545376346 DGTL CONTENT\nFACEBK *META VERIFIE...,FACEBK *META VERIFIEFB.ME/CC CA,1 HACKER WAY,MENLO PARK\nCA,94025,UNITED STATES,'320250120231674656',Business Services-Office Supplies,AMEXPLATINUM,54820.0


In [52]:
df.info

<bound method DataFrame.info of            Date                                 Description    Amount  \
0    2025-01-11                  ONLINE PAYMENT - THANK YOU -12021.51   
1    2025-01-11  ADOBE SYSTEMS Adobe SAN JOSE            CA     21.64   
2    2025-01-11  GOOGLE *ISHARINGSOFT G.CO/HELPPAY#      CA     16.04   
3    2025-01-12                                        UBER     28.32   
4    2025-01-12  FACEBK *META VERIFIEFB.ME/CC            CA     11.99   
...         ...                                         ...       ...   
1805 2025-10-24  PRIME VIDEO *NU6F775888-802-3080        WA      2.99   
1806 2025-10-25  PRIME VIDEO *NU8SR16888-802-3080        WA     14.99   
1807 2025-10-25  THE CLEANERS IO, LLCNEW YORK            NY     74.94   
1808 2025-10-25  UBER EATS           help.uber.com       CA    177.83   
1809 2025-10-25                                        UBER     53.01   

                                       Extended Details  \
0                            ONL

In [53]:
df.shape

(1810, 13)

Text Normalization Function : 

In [55]:
def normalize_text(text):
    if text is None:
        return ""
    text = str(text).lower()
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text


Preprocess Data : 

In [56]:
# Columns to combine for LLM input
TEXT_COLUMNS = ["Description", "Amount", "Extended Details", "Appears On Your Statement As", "Address", "City/State", "Country","Category"]

# Normalize text columns
for col in TEXT_COLUMNS:
    if col in df.columns:
        df[col] = df[col].apply(normalize_text)

# Combine features into a single text column
def combine_features(row):
    combined = ""
    for col in TEXT_COLUMNS:
        combined += " " + row.get(col, "")
    combined += " " + str(row.get("amount", ""))
    combined += " " + str(row.get("country", ""))
    return combined.strip()

df["combined_text"] = df.apply(combine_features, axis=1)
df.head()


,Date,Description,Amount,Extended Details,Appears On Your Statement As,Address,City/State,Zip Code,Country,Reference,Category,CC Name,G/L Account No,combined_text
0,2025-01-11,online payment thank you,12021 51,online payment thank you,online payment thank you,nan,nan,NaN,nan,'320250110186972138',nan,DELTARESERVE,NaN,online payment thank you 12021 51 online payme...
1,2025-01-11,adobe systems adobe san jose ca,21 64,567960456 www adobe com adobe systems adobe sy...,adobe systems adobe san jose ca,801 n 34th st,seattle wa,98103,united states,'320250120216930217',merchandise supplies mail order,AMEXPLATINUM,64100.0,adobe systems adobe san jose ca 21 64 56796045...
2,2025-01-11,google isharingsoft g co helppay ca,16 04,a0ywkmvs digital goods apps google isharingsof...,google isharingsoft g co helppay ca,1600 amphitheatre pkwy,mountain view ca,94043-1351,united states,320250120218752716',business services internet services,DELTARESERVE,64100.0,google isharingsoft g co helppay ca 16 04 a0yw...
3,2025-01-12,uber,28 32,fg9qdj8x lm5cypcr 10013 uber trip help uber co...,uber trip help uber com ca,1455 market st 4th floor,san francisco ca,94103,united states,'320250120213109403',transportation taxis coach,DELTARESERVE,61130.0,uber 28 32 fg9qdj8x lm5cypcr 10013 uber trip h...
4,2025-01-12,facebk meta verifiefb me cc ca,11 99,92545376346 dgtl content facebk meta verified ...,facebk meta verifiefb me cc ca,1 hacker way,menlo park ca,94025,united states,'320250120231674656',business services office supplies,AMEXPLATINUM,54820.0,facebk meta verifiefb me cc ca 11 99 925453763...


LLM Classification Function: 


In [57]:
PROMPT_TEMPLATE = """
You are an expert accounting assistant.

Classify the following expense into the correct category.
Also give a second possible category if applicable, with a short explanation.

Expense description:
{expense_text}

Return ONLY JSON in the format:
{{
    "Predicted_Category": "category name",
    "Confidence_Score": float between 0-1,
    "Alternative_Category": "category: explanation"
}}

Available categories: {categories}
"""

def classify_expense_llm(expense_text):
    prompt = PROMPT_TEMPLATE.format(expense_text=expense_text, categories=", ".join(CATEGORIES))
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )
        text_response = response["choices"][0]["message"]["content"]
        result = json.loads(text_response)
        return result
    except Exception as e:
        # fallback in case LLM output fails
        return {
            "Predicted_Category": "Unknown",
            "Confidence_Score": 0.0,
            "Alternative_Category": "Unknown: parsing error"
        }


Apply LLM to the DataFrame: 

In [58]:
df1=df.copy()

In [59]:
df1.head()

,Date,Description,Amount,Extended Details,Appears On Your Statement As,Address,City/State,Zip Code,Country,Reference,Category,CC Name,G/L Account No,combined_text
0,2025-01-11,online payment thank you,12021 51,online payment thank you,online payment thank you,nan,nan,NaN,nan,'320250110186972138',nan,DELTARESERVE,NaN,online payment thank you 12021 51 online payme...
1,2025-01-11,adobe systems adobe san jose ca,21 64,567960456 www adobe com adobe systems adobe sy...,adobe systems adobe san jose ca,801 n 34th st,seattle wa,98103,united states,'320250120216930217',merchandise supplies mail order,AMEXPLATINUM,64100.0,adobe systems adobe san jose ca 21 64 56796045...
2,2025-01-11,google isharingsoft g co helppay ca,16 04,a0ywkmvs digital goods apps google isharingsof...,google isharingsoft g co helppay ca,1600 amphitheatre pkwy,mountain view ca,94043-1351,united states,320250120218752716',business services internet services,DELTARESERVE,64100.0,google isharingsoft g co helppay ca 16 04 a0yw...
3,2025-01-12,uber,28 32,fg9qdj8x lm5cypcr 10013 uber trip help uber co...,uber trip help uber com ca,1455 market st 4th floor,san francisco ca,94103,united states,'320250120213109403',transportation taxis coach,DELTARESERVE,61130.0,uber 28 32 fg9qdj8x lm5cypcr 10013 uber trip h...
4,2025-01-12,facebk meta verifiefb me cc ca,11 99,92545376346 dgtl content facebk meta verified ...,facebk meta verifiefb me cc ca,1 hacker way,menlo park ca,94025,united states,'320250120231674656',business services office supplies,AMEXPLATINUM,54820.0,facebk meta verifiefb me cc ca 11 99 925453763...


In [19]:
# results = []

# # Loop through rows
# for idx, row in df.iterrows():
#     expense_text = row["combined_text"]
#     result = classify_expense_llm(expense_text)
#     results.append(result)

# # Add results to DataFrame
# df["Predicted_Category"] = [r["Predicted_Category"] for r in results]
# df["Confidence_Score"] = [r["Confidence_Score"] for r in results]
# df["Alternative_Category"] = [r["Alternative_Category"] for r in results]

# df.head()


In [60]:
# Create a new DataFrame for the first 100 rows
df1_100 = df1.head(100).copy()

# Loop and classify
results = []
for idx, row in df1_100.iterrows():
    expense_text = row["combined_text"]
    result = classify_expense_llm(expense_text)
    results.append(result)

# Add results to the new DataFrame
df1_100["Predicted_Category"] = [r["Predicted_Category"] for r in results]
df1_100["Confidence_Score"] = [r["Confidence_Score"] for r in results]
df1_100["Alternative_Category"] = [r["Alternative_Category"] for r in results]


In [61]:
df1_100.head()

,Date,Description,Amount,Extended Details,Appears On Your Statement As,Address,City/State,Zip Code,Country,Reference,Category,CC Name,G/L Account No,combined_text,Predicted_Category,Confidence_Score,Alternative_Category
0,2025-01-11,online payment thank you,12021 51,online payment thank you,online payment thank you,nan,nan,NaN,nan,'320250110186972138',nan,DELTARESERVE,NaN,online payment thank you 12021 51 online payme...,Unknown,0.0,Unknown: parsing error
1,2025-01-11,adobe systems adobe san jose ca,21 64,567960456 www adobe com adobe systems adobe sy...,adobe systems adobe san jose ca,801 n 34th st,seattle wa,98103,united states,'320250120216930217',merchandise supplies mail order,AMEXPLATINUM,64100.0,adobe systems adobe san jose ca 21 64 56796045...,Unknown,0.0,Unknown: parsing error
2,2025-01-11,google isharingsoft g co helppay ca,16 04,a0ywkmvs digital goods apps google isharingsof...,google isharingsoft g co helppay ca,1600 amphitheatre pkwy,mountain view ca,94043-1351,united states,320250120218752716',business services internet services,DELTARESERVE,64100.0,google isharingsoft g co helppay ca 16 04 a0yw...,Unknown,0.0,Unknown: parsing error
3,2025-01-12,uber,28 32,fg9qdj8x lm5cypcr 10013 uber trip help uber co...,uber trip help uber com ca,1455 market st 4th floor,san francisco ca,94103,united states,'320250120213109403',transportation taxis coach,DELTARESERVE,61130.0,uber 28 32 fg9qdj8x lm5cypcr 10013 uber trip h...,Unknown,0.0,Unknown: parsing error
4,2025-01-12,facebk meta verifiefb me cc ca,11 99,92545376346 dgtl content facebk meta verified ...,facebk meta verifiefb me cc ca,1 hacker way,menlo park ca,94025,united states,'320250120231674656',business services office supplies,AMEXPLATINUM,54820.0,facebk meta verifiefb me cc ca 11 99 925453763...,Unknown,0.0,Unknown: parsing error


## Other ways:

In [99]:
import pandas as pd

# Load the Excel file
file_path = "expenses.xlsx"
data = pd.read_excel(file_path)

# Select relevant columns
TEXT_COLUMNS = ["Description", "Amount", "Extended Details", "Appears On Your Statement As", "Address", "City/State", "Country"]
data = data[TEXT_COLUMNS]

# Drop rows with missing values in key columns
data = data.dropna(subset=["Description", "Amount"])

# Display the first few rows
print("Sample data:")
print(data.head())


Sample data:
                                  Description    Amount  \
0                  ONLINE PAYMENT - THANK YOU -12021.51   
1  ADOBE SYSTEMS Adobe SAN JOSE            CA     21.64   
2  GOOGLE *ISHARINGSOFT G.CO/HELPPAY#      CA     16.04   
3                                        UBER     28.32   
4  FACEBK *META VERIFIEFB.ME/CC            CA     11.99   

                                    Extended Details  \
0                         ONLINE PAYMENT - THANK YOU   
1  567960456   www.adobe.com\nADOBE SYSTEMS Adobe...   
2  A0YWKMVS    DIGITAL GOODS: APPS\nGOOGLE *ISHAR...   
3  FG9QDJ8X   LM5CYPCR         10013\nUber Trip\n...   
4  92545376346 DGTL CONTENT\nFACEBK *META VERIFIE...   

                 Appears On Your Statement As                    Address  \
0                  ONLINE PAYMENT - THANK YOU                        NaN   
1  ADOBE SYSTEMS Adobe SAN JOSE            CA              801 N 34TH ST   
2  GOOGLE *ISHARINGSOFT G.CO/HELPPAY#      CA     1600 AMPHITHEATRE

In [90]:
# Define GAAP categories (customize as needed)
gaap_categories = [
    "Travel", "Meals & Entertainment", "Office Supplies",
    "Utilities", "Rent", "Salaries", "Marketing", "Other"
]

# Prompt template for LLM classification with confidence and alternatives
prompt_template = """
Analyze the following expense and classify it into one of the GAAP categories: {categories}.
Return your response in the following JSON format:
{{
    "predicted_category": "your_prediction",
    "confidence_score": 0.0,  # Float between 0 and 1
    "alternative_category": "alternative_prediction",
    "reasoning": "brief_explanation"
}}

Expense details:
- Description: {description}
- Amount: ${amount:.2f}
- Extended Details: {extended_details}
- Appears On Your Statement As: {statement_as}
- Address: {address}
- City/State: {city_state}
- Country: {country}
"""


In [95]:
gaap_categories = [
    "Advertising & Marketing", "Bank Fees & Charges", "Commissions & Fees",
    "Consulting & Professional Fees", "Depreciation & Amortization",
    "Dues & Subscriptions", "Equipment Rental", "Insurance",
    "Legal & Accounting Fees", "Maintenance & Repairs", "Meals & Entertainment",
    "Office Supplies", "Postage & Shipping", "Rent & Lease", "Salaries & Wages",
    "Taxes & Licenses", "Telephone & Internet", "Travel & Lodging", "Utilities",
    "Direct Labor", "Direct Materials", "Freight & Shipping (for inventory)",
    "Manufacturing Overhead", "Interest Expense", "Loss on Sale of Assets",
    "Other Non-Operating Expenses", "Capital Equipment Purchases",
    "Software & Technology Purchases", "Charitable Contributions", "Miscellaneous Expenses"
]

prompt_template = f"""
Analyze the following expense and classify it into one of the GAAP categories: {gaap_categories}.
Return your response in the following JSON format:
{{
    "predicted_category": "your_prediction",
    "confidence_score": 0.0,  # Float between 0 and 1
    "alternative_category": "alternative_prediction",
    "reasoning": "brief_explanation"
}}

Expense details:
- Description: {{description}}
- Amount: ${{amount:.2f}}
- Extended Details: {{extended_details}}
- Appears On Your Statement As: {{statement_as}}
- Address: {{address}}
- City/State: {{city_state}}
- Country: {{country}}
"""


In [97]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import json

# Load API key
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

def classify_with_openai(row):
    prompt = prompt_template.format(
        categories=", ".join(gaap_categories),
        description=row["Description"],
        amount=row["Amount"],
        extended_details=row["Extended Details"],
        statement_as=row["Appears On Your Statement As"],
        address=row["Address"],
        city_state=row["City/State"],
        country=row["Country"]
    )
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=200
    )
    return json.loads(response.choices[0].message.content.strip())




In [98]:
# Test classification
test_expense = data.iloc[1]
result = classify_with_openai(test_expense)
print(result)

KeyError: '\n    "predicted_category"'

GOOD:

In [100]:
gaap_categories = [
    "Advertising & Marketing", "Bank Fees & Charges", "Commissions & Fees",
    "Consulting & Professional Fees", "Depreciation & Amortization",
    "Dues & Subscriptions", "Equipment Rental", "Insurance",
    "Legal & Accounting Fees", "Maintenance & Repairs", "Meals & Entertainment",
    "Office Supplies", "Postage & Shipping", "Rent & Lease", "Salaries & Wages",
    "Taxes & Licenses", "Telephone & Internet", "Travel & Lodging", "Utilities",
    "Direct Labor", "Direct Materials", "Freight & Shipping (for inventory)",
    "Manufacturing Overhead", "Interest Expense", "Loss on Sale of Assets",
    "Other Non-Operating Expenses", "Capital Equipment Purchases",
    "Software & Technology Purchases", "Charitable Contributions", "Miscellaneous Expenses"
]

# Remove the 'f' prefix and keep {categories} as a placeholder
prompt_template = """
Analyze the following expense and classify it into one of the GAAP categories: {categories}.
Return your response in the following JSON format:
{{
    "predicted_category": "your_prediction",
    "confidence_score": 0.0,  # Float between 0 and 1
    "alternative_category": "alternative_prediction",
    "reasoning": "brief_explanation"
}}

Expense details:
- Description: {description}
- Amount: ${amount:.2f}
- Extended Details: {extended_details}
- Appears On Your Statement As: {statement_as}
- Address: {address}
- City/State: {city_state}
- Country: {country}
"""

# Your classify_with_openai function remains the same
def classify_with_openai(row):
    prompt = prompt_template.format(
        categories=", ".join(gaap_categories),  # This will fill the {categories} placeholder
        description=row["Description"],
        amount=row["Amount"],
        extended_details=row["Extended Details"],
        statement_as=row["Appears On Your Statement As"],
        address=row["Address"],
        city_state=row["City/State"],
        country=row["Country"]
    )
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=200
    )
    return json.loads(response.choices[0].message.content.strip())

In [101]:
# Test classification
test_expense = data.iloc[0]
result = classify_with_openai(test_expense)
print(result)

{'predicted_category': 'Bank Fees & Charges', 'confidence_score': 0.9, 'alternative_category': 'Miscellaneous Expenses', 'reasoning': "The expense is most likely a bank fee or charge related to an online payment transaction, which falls under Bank Fees & Charges category. The description 'ONLINE PAYMENT - THANK YOU' suggests it is a fee charged by the bank for processing the online payment."}


More robust way:

In [102]:
import pandas as pd
import json
import os
from openai import OpenAI
from dotenv import load_dotenv
import time

# Load API key
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

# Define GAAP categories
gaap_categories = [
    "Advertising & Marketing", "Bank Fees & Charges", "Commissions & Fees",
    "Consulting & Professional Fees", "Depreciation & Amortization",
    "Dues & Subscriptions", "Equipment Rental", "Insurance",
    "Legal & Accounting Fees", "Maintenance & Repairs", "Meals & Entertainment",
    "Office Supplies", "Postage & Shipping", "Rent & Lease", "Salaries & Wages",
    "Taxes & Licenses", "Telephone & Internet", "Travel & Lodging", "Utilities",
    "Direct Labor", "Direct Materials", "Freight & Shipping (for inventory)",
    "Manufacturing Overhead", "Interest Expense", "Loss on Sale of Assets",
    "Other Non-Operating Expenses", "Capital Equipment Purchases",
    "Software & Technology Purchases", "Charitable Contributions", "Miscellaneous Expenses"
]

# Prompt template
prompt_template = """
Analyze the following expense and classify it into one of the GAAP categories: {categories}.
Return your response ONLY as a valid JSON object in this exact format:
{{
    "predicted_category": "your_prediction",
    "confidence_score": 0.0,
    "alternative_category": "alternative_prediction",
    "reasoning": "brief_explanation"
}}

Expense details:
- Description: {description}
- Amount: ${amount}
- Extended Details: {extended_details}
- Appears On Your Statement As: {statement_as}
- Address: {address}
- City/State: {city_state}
- Country: {country}
"""



In [103]:
def classify_expense(row):
    """Classify a single expense using GPT"""
    try:
        prompt = prompt_template.format(
            categories=", ".join(gaap_categories),
            description=str(row.get("Description", "")),
            amount=f"{row.get('Amount', 0):.2f}",
            extended_details=str(row.get("Extended Details", "")),
            statement_as=str(row.get("Appears On Your Statement As", "")),
            address=str(row.get("Address", "")),
            city_state=str(row.get("City/State", "")),
            country=str(row.get("Country", ""))
        )
        
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a financial classification assistant. Return valid JSON."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.1,
            max_tokens=300
        )
        
        response_text = response.choices[0].message.content.strip()
        
        # Extract JSON from response
        start_idx = response_text.find('{')
        end_idx = response_text.rfind('}') + 1
        json_str = response_text[start_idx:end_idx] if start_idx != -1 else response_text
        
        result = json.loads(json_str)
        
        return result
        
    except Exception as e:
        print(f"Error: {e}")
        return {
            "predicted_category": "Error",
            "confidence_score": 0.0,
            "alternative_category": "Error",
            "reasoning": f"Failed: {str(e)}"
        }



In [104]:
def process_excel_file(input_path, output_path=None, max_rows=None):
    """
    Main function to process Excel file
    """
    # Load Excel file
    df = pd.read_excel(input_path)
    print(f"Loaded {len(df)} rows from {input_path}")
    
    # Check required columns
    required = ["Description", "Amount", "Extended Details", "Appears On Your Statement As", 
                "Address", "City/State", "Country"]
    
    missing = [col for col in required if col not in df.columns]
    if missing:
        print(f"Warning: Missing columns: {missing}")
        for col in missing:
            df[col] = ""
    
    # Limit rows if specified
    if max_rows and max_rows < len(df):
        df = df.head(max_rows)
        print(f"Processing only first {max_rows} rows")
    
    # Initialize new columns
    df["predicted_category"] = ""
    df["confidence_score"] = 0.0
    df["alternative_category"] = ""
    
    # Process each row
    for idx, row in df.iterrows():
        print(f"Processing row {idx + 1}/{len(df)}...")
        
        result = classify_expense(row)
        
        df.at[idx, "predicted_category"] = result["predicted_category"]
        df.at[idx, "confidence_score"] = float(result["confidence_score"])
        df.at[idx, "alternative_category"] = result["alternative_category"]
        
        # Add delay to avoid rate limits
        time.sleep(0.1)
    
    # Save results
    if output_path:
        df.to_excel(output_path, index=False)
        print(f"Saved results to {output_path}")
    
    # Show summary
    print("\n=== Classification Summary ===")
    print(f"Top categories:")
    print(df["predicted_category"].value_counts().head(5))
    print(f"\nAverage confidence: {df['confidence_score'].mean():.2f}")
    
    return df


In [105]:
# ========== USAGE ==========

# 1. Set your file paths
input_file = "expenses.xlsx"  # Change this to your file
output_file = "classified_expenses.xlsx"  # Output file name

# 2. Process the file (set max_rows=5 for testing)
result_df = process_excel_file(
    input_path=input_file,
    output_path=output_file,
    max_rows=None  # Set to number if you want to limit
)


Loaded 1810 rows from expenses.xlsx
Processing row 1/1810...
Processing row 2/1810...
Processing row 3/1810...
Processing row 4/1810...
Processing row 5/1810...
Processing row 6/1810...
Processing row 7/1810...
Processing row 8/1810...
Processing row 9/1810...
Processing row 10/1810...
Processing row 11/1810...
Processing row 12/1810...
Processing row 13/1810...
Processing row 14/1810...
Processing row 15/1810...
Processing row 16/1810...
Processing row 17/1810...
Processing row 18/1810...
Processing row 19/1810...
Processing row 20/1810...
Processing row 21/1810...
Processing row 22/1810...
Processing row 23/1810...
Processing row 24/1810...
Processing row 25/1810...
Processing row 26/1810...
Processing row 27/1810...
Processing row 28/1810...
Processing row 29/1810...
Processing row 30/1810...
Processing row 31/1810...
Processing row 32/1810...
Processing row 33/1810...
Processing row 34/1810...
Processing row 35/1810...
Processing row 36/1810...
Processing row 37/1810...
Processing 

In [ ]:
# It takes 83 mins

In [106]:
# 3. Display results
print("\nFirst 5 classified rows:")
display(result_df[["Description", "predicted_category", "confidence_score", "alternative_category"]].head())


First 5 classified rows:


,Description,predicted_category,confidence_score,alternative_category
0,ONLINE PAYMENT - THANK YOU,Bank Fees & Charges,0.9,Miscellaneous Expenses
1,ADOBE SYSTEMS Adobe SAN JOSE CA,Software & Technology Purchases,0.9,Advertising & Marketing
2,GOOGLE *ISHARINGSOFT G.CO/HELPPAY# CA,Software & Technology Purchases,0.9,Advertising & Marketing
3,UBER,Travel & Lodging,0.9,Meals & Entertainment
4,FACEBK *META VERIFIEFB.ME/CC CA,Advertising & Marketing,0.9,Bank Fees & Charges


In [107]:
display(result_df[["Description", "predicted_category", "confidence_score", "alternative_category"]].head(40))

,Description,predicted_category,confidence_score,alternative_category
0,ONLINE PAYMENT - THANK YOU,Bank Fees & Charges,0.90,Miscellaneous Expenses
1,ADOBE SYSTEMS Adobe SAN JOSE CA,Software & Technology Purchases,0.90,Advertising & Marketing
2,GOOGLE *ISHARINGSOFT G.CO/HELPPAY# CA,Software & Technology Purchases,0.90,Advertising & Marketing
3,UBER,Travel & Lodging,0.90,Meals & Entertainment
4,FACEBK *META VERIFIEFB.ME/CC CA,Advertising & Marketing,0.90,Bank Fees & Charges
5,WEGMANS ASTOR PLACE NEW YORK NY,Meals & Entertainment,0.80,Office Supplies
6,ONLINE PAYMENT - THANK YOU,Bank Fees & Charges,0.90,Miscellaneous Expenses
7,LINKEDIN SN P3704854MOUNTAIN VIEW,Dues & Subscriptions,0.90,Advertising & Marketing
8,"IN *HEIDI KLING, PHDNEW YORK NY",Consulting & Professional Fees,0.90,Legal & Accounting Fees
9,PATREON* MEMBERSHIP SAN FRANCISCO CA,Dues & Subscriptions,0.90,Miscellaneous Expenses


In [108]:
# ONE-LINER: Create 100-row test files
import pandas as pd
df_ = pd.read_excel("expenses.xlsx")
[df_.iloc[i:i+100].to_excel(f"test_file_{i//100+1}.xlsx", index=False) for i in range(0, len(df_), 100)]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

## Using ollama

In [74]:
import ollama
import json

def classify_with_ollama(row):
    prompt = prompt_template.format(
        categories=", ".join(gaap_categories),
        description=row["Description"],
        amount=row["Amount"],
        extended_details=row["Extended Details"],
        statement_as=row["Appears On Your Statement As"],
        address=row["Address"],
        city_state=row["City/State"],
        country=row["Country"]
    )
    response = ollama.chat(
        model="llama2",
        messages=[{"role": "user", "content": prompt}]
    )
    return json.loads(response["message"]["content"].strip())




In [75]:
# Test classification
test_expense = data.iloc[0]
result = classify_with_ollama(test_expense)
print(result)

ConnectionError: Failed to connect to Ollama. Please check that Ollama is downloaded, running and accessible. https://ollama.com/download